# RusTorch CoreML 集成 - Rust 内核

这个笔记本演示了如何在 RusTorch 中使用 CoreML。
运行在 Rust 内核上。

## 检查所需依赖项和功能

In [ ]:
// RusTorch 基本使用
extern crate rustorch;

use rustorch::tensor::Tensor;
use rustorch::gpu::DeviceType;

println!("RusTorch 版本: {}", env!("CARGO_PKG_VERSION"));
println!("Rust 版本: {}", env!("RUSTC_VERSION"));

## 检查 CoreML 可用性

检查当前系统上是否可用 CoreML。

In [ ]:
#[cfg(any(feature = "coreml", feature = "coreml-hybrid", feature = "coreml-fallback"))]
{
    use rustorch::backends::DeviceManager;
    
    let coreml_available = DeviceManager::is_coreml_available();
    println!("CoreML 可用: {}", coreml_available);
    
    if coreml_available {
        println!("🎉 CoreML 可用！");
        println!("平台: macOS");
        
        // 显示设备信息
        use rustorch::gpu::coreml::device_cache::DeviceCache;
        let cache = DeviceCache::global();
        cache.warmup();
        
        let stats = cache.get_stats();
        println!("缓存统计: {:?}", stats);
    } else {
        println!("⚠️ CoreML 不可用");
        println!("请使用 CPU 或其他 GPU 后端");
    }
}

#[cfg(not(any(feature = "coreml", feature = "coreml-hybrid", feature = "coreml-fallback")))]
{
    println!("❌ CoreML 功能未启用");
    println!("请使用 --features coreml 构建");
}

## 基本张量操作

In [ ]:
// 创建基本张量
let a = Tensor::zeros(&[2, 3]);
let b = Tensor::ones(&[3, 2]);

println!("张量 A 形状: {:?}", a.shape());
println!("张量 B 形状: {:?}", b.shape());

// 基本矩阵乘法
let result = a.matmul(&b);
println!("结果形状: {:?}", result.shape());
println!("基本张量操作完成");

## CoreML 设备操作

In [ ]:
#[cfg(any(feature = "coreml", feature = "coreml-hybrid", feature = "coreml-fallback"))]
{
    use rustorch::gpu::coreml::{CoreMLDevice, CoreMLBackend};
    use rustorch::backends::BackendConfig;
    
    // 尝试创建 CoreML 设备
    match CoreMLDevice::new(0) {
        Ok(device) => {
            println!("🖥️ CoreML 设备创建成功");
            println!("设备 ID: {}", device.id());
            println!("可用: {}", device.is_available());
            println!("内存限制: {} MB", device.memory_limit() / (1024 * 1024));
            
            // 创建后端配置
            let config = BackendConfig::new()
                .with_caching(true)
                .with_max_cache_size(200)
                .with_profiling(true)
                .with_auto_fallback(true);
            
            println!("⚙️ 后端配置: {:?}", config);
            
            // 创建 CoreML 后端
            match CoreMLBackend::new(device, config) {
                Ok(backend) => {
                    println!("🚀 CoreML 后端已初始化");
                    
                    // 获取统计信息
                    let stats = backend.get_statistics();
                    println!("📊 后端统计:");
                    println!("  总操作数: {}", stats.total_operations);
                    println!("  缓存命中: {}", stats.cache_hits);
                    println!("  缓存未命中: {}", stats.cache_misses);
                    println!("  回退操作: {}", stats.fallback_operations);
                    
                    // 在 CoreML 上创建张量
                    let tensor_a = Tensor::randn(&[64, 64]).to_device(&backend);
                    let tensor_b = Tensor::randn(&[64, 64]).to_device(&backend);
                    
                    println!("📐 在 CoreML 设备上创建张量");
                    
                    // 矩阵乘法操作
                    let start = std::time::Instant::now();
                    let result = tensor_a.matmul(&tensor_b);
                    let duration = start.elapsed();
                    
                    println!("✅ 矩阵乘法完成");
                    println!("⏱️ 执行时间: {:?}", duration);
                    println!("🎯 结果形状: {:?}", result.shape());
                    
                    // 清理缓存
                    backend.cleanup_cache();
                    println!("🧹 缓存已清理");
                }
                Err(e) => println!("❌ 创建 CoreML 后端错误: {:?}", e),
            }
        }
        Err(e) => {
            println!("❌ 创建 CoreML 设备错误: {:?}", e);
            println!("此系统上可能无法使用 CoreML");
        }
    }
}

#[cfg(not(any(feature = "coreml", feature = "coreml-hybrid", feature = "coreml-fallback")))]
{
    println!("⚠️ 跳过 CoreML 操作 - 功能未启用");
}

## 性能比较

In [ ]:
use std::time::Instant;

fn benchmark_operations() {
    let sizes = vec![(64, 64), (128, 128), (256, 256), (512, 512)];
    
    println!("🏁 操作基准测试:");
    println!("大小\t\tCPU (毫秒)\t首选设备");
    println!("-" * 40);
    
    for (rows, cols) in sizes {
        // 在 CPU 上创建张量
        let a = Tensor::randn(&[rows, cols]);
        let b = Tensor::randn(&[cols, rows]);
        
        // 测量 CPU 时间
        let start = Instant::now();
        let _result = a.matmul(&b);
        let cpu_duration = start.elapsed();
        
        // 确定首选设备
        let preferred_device = if rows * cols < 1000 {
            "CPU"
        } else if rows * cols < 10000 {
            "Metal GPU"
        } else {
            "CoreML"
        };
        
        println!("{}x{}\t\t{:.2}\t\t{}", 
                rows, cols, 
                cpu_duration.as_millis() as f64, 
                preferred_device);
    }
}

benchmark_operations();
println!("\n📝 注意：设备选择基于张量大小和可用性");

## 智能设备选择

In [ ]:
#[cfg(any(feature = "coreml", feature = "coreml-hybrid", feature = "coreml-fallback"))]
{
    use rustorch::backends::{DeviceManager, DeviceSelector};
    
    fn demonstrate_device_selection() {
        println!("🎯 智能设备选择演示:");
        
        let operations = vec![
            ("小乘法", vec![16, 16], "CPU"),
            ("2D 卷积", vec![32, 3, 224, 224], "CoreML"),
            ("矩阵变换", vec![128, 128], "Metal GPU"),
            ("大批次操作", vec![512, 512], "CoreML"),
            ("向量计算", vec![1000], "CPU"),
        ];
        
        for (name, shape, preferred) in operations {
            println!("  {:<25} {:?} -> {}", name, shape, preferred);
            
            // 模拟基于规则的选择
            let tensor_size: usize = shape.iter().product();
            let selected_device = match tensor_size {
                size if size < 1000 => DeviceType::Cpu,
                size if size < 50000 => DeviceType::MetalGpu,
                _ => {
                    if DeviceManager::is_coreml_available() {
                        DeviceType::CoreML
                    } else {
                        DeviceType::MetalGpu
                    }
                }
            };
            
            println!("    -> 选择的设备: {:?}", selected_device);
        }
        
        println!("\n📝 选择逻辑:");
        println!("  • < 1K 元素: CPU（最小开销）");
        println!("  • 1K-50K 元素: Metal GPU（平衡）");
        println!("  • > 50K 元素: CoreML（优化）或 Metal GPU（回退）");
    }
    
    demonstrate_device_selection();
}

#[cfg(not(any(feature = "coreml", feature = "coreml-hybrid", feature = "coreml-fallback")))]
{
    println!("⚠️ 设备选择演示已跳过 - CoreML 功能不可用");
}

## 高级示例：神经网络层

In [ ]:
fn simulate_neural_layer() {
    println!("🧠 神经网络层模拟:");
    
    // 层配置
    let batch_size = 32;
    let input_dim = 784;   // 28x28 MNIST
    let hidden_dim = 256;
    let output_dim = 10;   // 10个类别
    
    println!("📊 配置:");
    println!("  批次大小: {}", batch_size);
    println!("  输入维度: {}", input_dim);
    println!("  隐藏维度: {}", hidden_dim);
    println!("  输出维度: {}", output_dim);
    
    // 创建张量
    let input = Tensor::randn(&[batch_size, input_dim]);
    let weight1 = Tensor::randn(&[input_dim, hidden_dim]);
    let weight2 = Tensor::randn(&[hidden_dim, output_dim]);
    
    println!("\n🔄 前向传播:");
    
    // 模拟前向传播
    let start = Instant::now();
    
    // 层 1: 输入 -> 隐藏
    let hidden = input.matmul(&weight1);
    println!("  ✅ 输入 -> 隐藏: {:?}", hidden.shape());
    
    // ReLU 激活函数（模拟）
    let activated = hidden.relu();
    println!("  ✅ ReLU 激活已应用");
    
    // 层 2: 隐藏 -> 输出
    let output = activated.matmul(&weight2);
    println!("  ✅ 隐藏 -> 输出: {:?}", output.shape());
    
    let total_time = start.elapsed();
    
    println!("\n⏱️ 总前向传播时间: {:?}", total_time);
    println!("🚀 估计性能: {:.0} 样本/秒", 
             (batch_size as f64) / total_time.as_secs_f64());
    
    println!("\n📝 在实际实现中:");
    println!("  • 大矩阵将使用 CoreML");
    println!("  • 激活将使用 Metal GPU");
    println!("  • 小操作将保留在 CPU 上");
}

simulate_neural_layer();

## 错误处理和回退

In [ ]:
fn demonstrate_fallback_behavior() {
    println!("🔄 回退行为演示:");
    
    // 模拟可能在 CoreML 上失败的操作
    let complex_tensor = Tensor::randn(&[100, 100]);
    
    println!("🎯 尝试 CoreML 操作...");
    
    // 在实际实现中，这将是:
    // match tensor.to_coreml() {
    //     Ok(coreml_tensor) => { /* 使用 CoreML */ },
    //     Err(_) => { /* 回退到 Metal/CPU */ }
    // }
    
    let use_coreml = false; // 模拟 CoreML 失败
    
    if use_coreml {
        println!("✅ CoreML 操作成功");
    } else {
        println!("⚠️ CoreML 不可用，使用回退");
        
        // 回退到 Metal GPU
        let start = Instant::now();
        let result = complex_tensor.matmul(&complex_tensor);
        let fallback_time = start.elapsed();
        
        println!("✅ 回退操作完成");
        println!("⏱️ 回退时间: {:?}", fallback_time);
        println!("📐 结果形状: {:?}", result.shape());
    }
    
    println!("\n📝 回退策略:");
    println!("  1. 尝试 CoreML（最佳性能）");
    println!("  2. 回退到 Metal GPU（良好兼容性）");
    println!("  3. 最终回退到 CPU（最大兼容性）");
}

demonstrate_fallback_behavior();

## 总结和后续步骤

In [ ]:
println!("📋 RusTorch CoreML 集成总结（Rust 内核）:");
println!();
println!("✅ 演示的功能:");
println!("  • CoreML 可用性检查");
println!("  • 设备创建和管理");
println!("  • 后端配置");
println!("  • 基本张量操作");
println!("  • 性能基准测试");
println!("  • 智能设备选择");
println!("  • 回退行为");
println!();
println!("🚧 开发领域:");
println!("  • 完整的 CoreML 操作实现");
println!("  • 内存传输优化");
println!("  • 扩展的张量类型支持");
println!("  • 详细的性能分析");
println!("  • 与 ML 管道的集成");
println!();
println!("🎯 推荐的后续步骤:");
println!("  1. 使用预训练的 CoreML 模型进行测试");
println!("  2. 与其他后端进行比较基准测试");
println!("  3. 针对特定用例进行优化");
println!("  4. 部署到生产应用程序");
println!();

#[cfg(any(feature = "coreml", feature = "coreml-hybrid", feature = "coreml-fallback"))]
{
    if rustorch::backends::DeviceManager::is_coreml_available() {
        println!("🎉 所有 CoreML 功能都可以测试！");
    } else {
        println!("⚠️ CoreML 已启用但在此系统上不可用");
    }
}

#[cfg(not(any(feature = "coreml", feature = "coreml-hybrid", feature = "coreml-fallback")))]
{
    println!("⚠️ 使用 CoreML 功能构建以获得完整功能");
}

println!("\n🚀 准备好使用 RusTorch 进行高级 CoreML 开发！");